In [1]:
import praw
from psaw import PushshiftAPI
import datetime as dt
import csv
import pandas as pd

In [2]:
#old code to grab tweet ids -- idk might be useful later
# for i in range(10, 32):
#     url = f"https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/2021-08-{i}/2021-08-{i}-dataset.tsv.gz?raw=true"
#     tweet = pd.read_csv(url, compression = 'gzip', delimiter = '\t')
#     tweet = tweet[tweet["lang"] == 'en']
#     print(tweet.shape)
#     tweet['tweet_id'].to_csv(f"C:\\Users\chris\\Downloads\\filtered_tweet_ids\\08_{i}.csv", index = False, header = False)

In [2]:
reddit = praw.Reddit(

api = PushshiftAPI()

list of subreddits:  
* antivaccine now a space jam sub?
* trueantivaccination (quarantined) done
* Vaccines done
* CovidVaccine done
* CovidVaccinated done
* AntiVaxxers done
* conspiracy* 
* conspiracytheories* 
* NoNewNormal banned :( = prob a lot less antivaxx posts
* conspiracy_commons* 
* COVID19* done
* COVID private
* coronavirus* done
* conservative* done
* worldnews* done
* news* 

\* = search terms used

In [3]:
covid_terms = 'covid|covid19|c19|corona|coronavirus|CoV-2|pandemic|variant|delta|virus|infection|infect|fauci|cdc|fda|mandate'
vaccine_terms = 'shot|vaccine|vacine|vacines|vaccines|vaccinate|vaccination|vaccinations|vaccinated|vaccinating|vaxxed|vaxx|vax|\
unvaccinated|unvaxxed|antivaxx|antivaccination|anti|\
moderna|pfizer|J&J|immune|immunize|immunizes|immunized|immunization|immunizations'

In [5]:
start_epoch=int(dt.datetime(2021, 8, 10).timestamp()) #searches for posts after midnight 8/10
end_epoch = int(dt.datetime(2021, 9, 1).timestamp()) #stops getting posts at midnight 9/1
ids = list(api.search_submissions(after=start_epoch, #makes list of psaw submission objects
                                  before = end_epoch,
                                  filter = ['id'],
                                  title = covid_terms+vaccine_terms,
                                  subreddit = 'news'))#searches for keywords in title only)
len(ids)

In [29]:
with open("C:\\Users\\chris\\Documents\\Math168Project\\no_search.csv", 'a', encoding="utf-8") as file:
    #setting up the csv file
    headers = ['ID', 'Date_utc', 'Author', 'Body', 'Submission', 'Parent_ID', 'Parent_Author', 'Level']
    writer = csv.DictWriter(file, fieldnames = headers, extrasaction='ignore')
    writer.writeheader()
    
    for psaw_submission in ids: #goes through list of ids matching criteria
        post = reddit.submission(id = psaw_submission.d_['id']) #turns psaw submission into praw submission
        post.comments.replace_more(limit = 0)
        
        for comment in post.comments: #goes through top level comments 
            if (not post.author) and (not comment.author): #skip if both post and comment author deleted
                continue
            data = {'ID':comment.id, 'Date_utc':comment.created_utc, 'Author':comment.author, 'Body': comment.body,
            'Submission':post.id, 'Parent_ID':post.id, 'Parent_Author':post.author, 'Level':1}
            writer.writerow(data)
            
            for reply in comment.replies: #second level comments
                if not reply.author: #account deleted/comment deleted/moderator removed
                    continue
                data = {'ID':reply.id, 'Date_utc':reply.created_utc, 'Author':reply.author, 'Body': reply.body,
                        'Submission':post.id, 'Parent_ID':comment.id, 'Parent_Author':comment.author, 'Level':2}
                writer.writerow(data)

In [20]:
# with open("C:\\Users\\chris\\Documents\\Math168Project\\conspiracy_subs.csv", 'a', encoding="utf-8") as file:
#     #setting up the csv file
#     headers = ['ID', 'Date_utc', 'Author', 'Body', 'Parent_ID', 'Parent_Author', 'Post_ID', 'Post_Title', 'Subreddit_ID']
#     writer = csv.DictWriter(file, fieldnames = headers, extrasaction='ignore')
#     writer.writeheader()
    
#     for psaw_submission in ids: #goes through list of ids matching criteria
#         post = reddit.submission(id = psaw_submission.d_['id']) #turns psaw submission into praw submission
       
#         post.comments.replace_more(limit = 0)
#         for comment in post.comments.list(): 
#             if not comment.author: #skip if comment author deleted
#                 continue
#             if comment.author == 'AutoModerator': #skip automatic comments
#                 continue
#             parent_author = reddit.comment(id = comment.parent_id).author
#             data = {'ID':comment.id, 'Date_utc':comment.created_utc, 'Author':comment.author, 'Body': comment.body,
#                     'Parent_ID':comment.parent_id, 'Parent_Author':parent_author, 
#                     'Post_ID':post.id, 'Post_Title': post.title,
#                    'Subreddit_ID':comment.subreddit_id}
#             writer.writerow(data)
            